In [14]:
import yahoo_fin as yf
from yahoo_fin.stock_info import get_data
import time
import datetime
import pandas as pd
from yahoo_fin import stock_info as si

import tensorflow as tf 
import numpy as np
from tensorflow.keras import layers

# getting the right tickers

some of this is shamelessly copying this:
https://levelup.gitconnected.com/how-to-get-all-stock-symbols-a73925c16a1b

things we did to clean data:

1. we removed the tickers with $.


2. we added exception handeling that tries similer strings to the input. (note that this is not implemented on the set itself but when fetching, leading to multiple requsts and potentialy to errors)


3. we have painstakingly went ahead and manualy found all exceptions and removed them if no reformating was possible.

In [15]:
df1 = pd.DataFrame( si.tickers_sp500() )
df2 = pd.DataFrame( si.tickers_nasdaq() )
df3 = pd.DataFrame( si.tickers_dow() )
df4 = pd.DataFrame( si.tickers_other() )

sym1 = set( symbol for symbol in df1[0].values.tolist() )
sym2 = set( symbol for symbol in df2[0].values.tolist() )
sym3 = set( symbol for symbol in df3[0].values.tolist() )
sym4 = set( symbol for symbol in df4[0].values.tolist() )

symbols = set.union( sym1, sym2, sym3, sym4 )
del(sym1, sym2, sym3, sym4,df1, df2, df3, df4)

symbols

{'DGRS',
 '',
 'VXUS',
 'STC',
 'DFEN',
 'MBI',
 'PONOU',
 'BENEW',
 'BXC',
 'EYLD',
 'EZPW',
 'SRL',
 'ACGLO',
 'GNK',
 'DCGOW',
 'EBET',
 'TRIP',
 'MPW',
 'VOYA',
 'BYD',
 'C$K',
 'MPACR',
 'LODE',
 'GGMCW',
 'PUCKU',
 'RJF$B',
 'HOUR',
 'CPLP',
 'SIER',
 'VHNA',
 'EFAX',
 'DLHC',
 'BDL',
 'MDH.W',
 'TCN',
 'SRG$A',
 'BCAT',
 'EQHA.W',
 'OEUR',
 'CRU',
 'ADRE',
 'AFIB',
 'SKYW',
 'STR.W',
 'DOG',
 'EVGRU',
 'BRACR',
 'FRON',
 'DEUS',
 'CENQW',
 'IMAQW',
 'DVND',
 'SAN',
 'CIG.C',
 'LEVI',
 'PHR',
 'KOLD',
 'PPG',
 'BUZZ',
 'MSBIP',
 'CVR',
 'SVFAW',
 'PNFP',
 'FLR',
 'BSAC',
 'RES',
 'CDC',
 'MMS',
 'MSFU',
 'XJH',
 'VMOT',
 'VXZ',
 'AVSF',
 'EVOP',
 'VYGG.W',
 'VRNS',
 'HAWX',
 'ACV',
 'B',
 'BOAC.U',
 'AHL$D',
 'EPR$E',
 'IGM',
 'STLV',
 'SDP',
 'NKTR',
 'TRUE',
 'IMPL',
 'CND.W',
 'SF$B',
 'TELL',
 'RCLFW',
 'FTXR',
 'OYST',
 'PDSB',
 'JHSC',
 'VERS',
 'RYJ',
 'CEQP',
 'NOCT',
 'PRPO',
 'ALCO',
 'GDS',
 'RORO',
 'AEHAU',
 'KBNTW',
 'PFHD',
 'SEA',
 'MPRA',
 'GGROW',
 'GBDC',
 'MGC

In [18]:
my_list = ['W', 'R', 'P', 'Q']
del_set = set()
sav_set = set()

for symbol in symbols:
    if (len( symbol ) > 4 and symbol[-1] in my_list or "$" in symbol):
        del_set.add( symbol )
    else:
        sav_set.add( symbol )
        
p=pd.read_csv("shitlist")
p=list(p["0"])
#p2=pd.read_csv("errors.csv")
#p.extend(list(p2["0"]))
del_set=del_set.union(p)
sav_set.difference_update(p)
print( f'Removed {len( del_set )} unqualified stock symbols...' )
print( f'There are {len( sav_set )} qualified stock symbols...' )

Removed 1592 unqualified stock symbols...
There are 10606 qualified stock symbols...


In [19]:
sav_set.discard("")
#this just drops the odd empty string on the start
ticker_list=list(sav_set)

# fetching data 

we are scraping yahoo finance for the data 

there are some repeat trys to get the right ticker format


.

I implemented some basic multithreading to speed this up 

its not much but it helps

In [20]:
start='01/01/2020' 
end='09/14/2022'

In [21]:
import concurrent.futures 

def get_seiries(ticker,start,end,entry="adjclose"):
    try:
        raw = get_data(ticker.split(".")[0], start_date=start, end_date=end)[entry]
    except: 
        try:
            raw = get_data(ticker.replace(".","-"), start_date=start, end_date=end)[entry]
        except:
            raw = get_data(ticker[0:4], start_date=start, end_date=end)[entry]
    return raw/raw[0]

def get_df(l,start,end,entry="adjclose"):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        x=[executor.submit(lambda x: get_seiries(x,start,end,entry=entry),name)for name in l]
        x=[a.result() for a in x]
    for i,c in enumerate(x):
        c.name=l[i]
    return pd.DataFrame(x).transpose()
y=get_df(ticker_list[2000:2100],start,end,entry="adjclose")
y

,FIZZ,UNMA,MINM,RETO,IDBA,GERN,CCEL,NTRSO,IPAY,MULN,...,FRNW,HIX,GPMT,BNNRU,LFTR,VBTX,SWK,TWND,UBOT,STTK
2020-01-02,1.000000,1.000000,1.000000,1.000000,NaN,1.000000,1.000000,1.000000,1.000000,1.000000,...,NaN,1.000000,1.000000,NaN,NaN,1.000000,1.000000,NaN,1.000000,NaN
2020-01-03,0.991152,1.001829,1.000000,1.000000,NaN,0.977444,1.005548,1.000768,0.991849,0.984756,...,NaN,1.002958,1.000000,NaN,NaN,1.001365,0.994644,NaN,0.965345,NaN
2020-01-06,0.994239,1.001463,1.086957,0.935080,NaN,0.984962,1.000000,0.998848,0.993439,1.027439,...,NaN,0.994083,1.003855,NaN,NaN,0.998294,0.991072,NaN,0.990455,NaN
2020-01-07,0.955144,1.000805,1.086957,0.935080,NaN,1.022556,1.011096,0.995776,0.990457,1.067073,...,NaN,0.994083,1.000551,NaN,NaN,0.986694,0.983157,NaN,0.981645,NaN
2020-01-08,0.964609,0.999634,1.043478,1.138952,NaN,1.000000,1.034674,0.998464,1.002982,1.060976,...,NaN,1.000000,1.000551,NaN,NaN,0.988400,0.987859,NaN,0.995595,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-07,2.374637,0.946205,0.260870,0.609339,0.230142,2.045113,0.923717,0.881757,0.845129,0.204268,...,0.980689,0.956779,0.568508,1.006061,0.998498,1.019602,0.538541,1.047817,1.128487,0.172610
2022-09-08,2.372296,0.946205,0.252174,0.650342,0.230142,2.052632,1.009709,0.869349,0.847515,0.185976,...,0.988421,0.958575,0.564671,1.006061,0.998498,1.021054,0.542050,1.044699,1.159430,0.181395
2022-09-09,2.197628,0.961962,0.278261,0.637813,0.230142,1.992481,0.962552,0.904917,0.871173,0.170732,...,0.997370,0.971140,0.581937,1.006061,0.998498,1.073667,0.550976,1.043659,1.228038,0.165891
2022-09-12,2.147990,0.971417,0.260870,0.637813,0.236170,2.090225,0.943135,0.922701,0.880716,0.176829,...,1.000118,0.981911,0.597285,1.005051,0.998999,1.097252,0.564827,1.041580,1.258888,0.168475


In [ ]:
def time_encode(t):
    first= [(a.day_of_week,a.day/30,a.month/12,(a.year-2020)) for a in t]
    return [[x[i] for x in first] for i in range(4)]
time_encode(y.index[0:5])

In [ ]:
look=layers.StringLookup(vocabulary=ticker_list)
inv_look=layers.StringLookup(vocabulary=ticker_list,invert=True)
tick_embed=layers.Embedding(len(ticker_list)+1,512,embeddings_constraint= tf.keras.constraints.unit_norm)

day_embed=layers.Embedding(7,10)

# secondry testing for errors

In [22]:
def tester_func(name,start=start,end=end,entry="adjclose"):
    try:
        c= get_seiries(name,start,end,entry)
        c.name=name
        return (True,c)
    except:
        return (False,name)

In [23]:
mini=ticker_list
errors=[]
data=[]
c=0
x=list(range(len(mini)))
with concurrent.futures.ThreadPoolExecutor() as executor:
    for i in range(len(mini)):
        x[i]=executor.submit(tester_func,mini[i])
    
    print("thread setup complete")
    
    for i in range(len(mini)):
        if(i%100==0):
            print(i)
        r=x[i].result()
        if (r[0]): 
            data.append(r[1])
        else:
            errors.append(r[1])
            c+=1
            print(f"error number {c}: name={errors[-1]}")

print("done")

thread setup complete
0
100
200
error number 1: name=CTEST
300
400
500
600
700
800
900
error number 2: name=BLLD
1000
error number 3: name=EMZA
1100
1200
1300
1400
1500
1600
error number 4: name=CTEST.V
1700
1800
1900
2000
2100
2200
error number 5: name=CTEST.E
2300
error number 6: name=ZBZZT
2400
2500
2600
error number 7: name=CTEST.O
2700
2800
2900
3000
error number 8: name=ZWZZT
3100
3200
3300
3400
3500
error number 9: name=CIRC
3600
3700
error number 10: name=EMCH
3800
3900
4000
4100
4200
error number 11: name=BSJU
4300
error number 12: name=ZXZZT
4400
error number 13: name=ATEST.A
4500
error number 14: name=CTEST.G
4600
error number 15: name=ZXYZ.A
4700
4800
error number 16: name=CBO
4900
5000
5100
error number 17: name=ATEST.B
5200
5300
error number 18: name=RCA
5400
error number 19: name=CTEST.L
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
error number 20: name=YEAR
6500
error number 21: name=ZVZZT
6600
6700
6800
6900
error number 22: name=TAFI
7000
7100
7200
error number 2

In [ ]:
#pd.DataFrame(errors).to_csv("errors.csv")

In [ ]:
get_data("BLLD", start_date=start)

In [ ]:
errors[0]

In [24]:
data=pd.concat(data)

In [25]:
pd.DataFrame(x).transpose()

2020-01-02    1.000000
2020-01-03    0.993360
2020-01-06    0.992032
2020-01-07    0.990172
2020-01-08    0.992032
                ...   
2022-09-07    1.078324
2022-09-08    1.129586
2022-09-09    1.156518
2022-09-12    1.165366
2022-09-13    1.136222
Length: 5871775, dtype: float64

In [26]:
len(data)

5871775

In [40]:
p=pd.read_csv("shitlist")
p=list(p["0"])
p.extend(errors)
p=set(p)
#p=list(p)
pd.DataFrame(p).to_csv("shitlist")